In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix
)
from sklearn.preprocessing import label_binarize
from scipy.stats import chi2


In [ ]:
dataset_path = "/kaggle/input/minor-dataset/Data_Minor"

classes = ["HR", "DR", "RVO"]
num_classes = len(classes)

img_size = (224, 224)
input_shape = (224, 224, 3)

batch_size = 8
epochs = 25


In [ ]:
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(2.0, (8,8))
    l = clahe.apply(l)
    return cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2RGB)

def preprocess_image(image, use_preprocessing=True):
    if use_preprocessing:
        image = apply_clahe(image)
    return image.astype(np.float32) / 255.0


In [ ]:
def load_images(use_preprocessing=True):
    images, labels = [], []
    for label, cls in enumerate(classes):
        folder = os.path.join(dataset_path, cls)
        for f in os.listdir(folder):
            img = cv2.imread(os.path.join(folder, f))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, img_size)
            img = preprocess_image(img, use_preprocessing)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

images, labels = load_images(use_preprocessing=True)

X_train, X_test, y_train, y_test = train_test_split(
    images, labels,
    test_size=0.2,
    stratify=labels,
    random_state=42
)


In [ ]:
def build_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, 3, activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
def transformer_block(x, num_heads=4, ff_dim=128):
    attn = layers.MultiHeadAttention(num_heads, key_dim=x.shape[-1])(x, x)
    x = layers.Add()([x, attn])
    x = layers.LayerNormalization()(x)

    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(x.shape[-1])(ff)
    x = layers.Add()([x, ff])
    return layers.LayerNormalization()(x)


In [ ]:
def build_transformer(input_shape, num_classes, patch_size=16, embed_dim=64):
    inputs = layers.Input(shape=input_shape)

    patches = layers.Conv2D(
        embed_dim,
        kernel_size=patch_size,
        strides=patch_size
    )(inputs)

    x = layers.Reshape((-1, embed_dim))(patches)

    for _ in range(2):
        x = transformer_block(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)


In [ ]:
def build_hybrid(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Reshape((-1, 64))(x)

    for _ in range(2):
        x = transformer_block(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)


In [ ]:
model = build_hybrid(input_shape, num_classes)

model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(
    X_train,
    to_categorical(y_train, num_classes),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)


In [ ]:
y_prob = model.predict(
    X_test,
    batch_size=4,   # or even 2 if needed
    verbose=1
)

y_pred = np.argmax(y_prob, axis=1)
y_true = y_test

acc = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {acc*100:.2f}%")


In [ ]:
y_true_bin = label_binarize(y_true, classes=[0,1,2])

auc_macro = roc_auc_score(
    y_true_bin,
    y_prob,
    average="macro",
    multi_class="ovr"
)

print(f"Macro AUC: {auc_macro:.4f}")


In [ ]:
plt.figure(figsize=(7,7))

for i, cls in enumerate(classes):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
    auc = roc_auc_score(y_true_bin[:, i], y_prob[:, i])
    plt.plot(fpr, tpr, label=f"{cls} (AUC={auc:.3f})")

plt.plot([0,1], [0,1], '--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves (OvR)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def expected_calibration_error(y_true, y_prob, n_bins=10):
    confidences = np.max(y_prob, axis=1)
    predictions = np.argmax(y_prob, axis=1)
    accuracies = (predictions == y_true)

    bins = np.linspace(0, 1, n_bins + 1)
    ece = 0.0

    for i in range(n_bins):
        mask = (confidences > bins[i]) & (confidences <= bins[i+1])
        if np.any(mask):
            bin_acc = np.mean(accuracies[mask])
            bin_conf = np.mean(confidences[mask])
            ece += np.abs(bin_acc - bin_conf) * np.mean(mask)

    return ece

ece = expected_calibration_error(y_true, y_prob)
print(f"ECE: {ece:.4f}")


In [ ]:
def plot_reliability(y_true, y_prob, n_bins=10):
    confidences = np.max(y_prob, axis=1)
    predictions = np.argmax(y_prob, axis=1)
    accuracies = (predictions == y_true)

    bins = np.linspace(0, 1, n_bins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2

    bin_accs, bin_confs = [], []

    for i in range(n_bins):
        mask = (confidences > bins[i]) & (confidences <= bins[i+1])
        if np.any(mask):
            bin_accs.append(np.mean(accuracies[mask]))
            bin_confs.append(np.mean(confidences[mask]))
        else:
            bin_accs.append(0)
            bin_confs.append(0)

    plt.figure(figsize=(6,6))
    plt.plot([0,1], [0,1], '--', color='gray')
    plt.bar(bin_centers, bin_accs, width=0.08, alpha=0.7, label="Accuracy")
    plt.plot(bin_centers, bin_confs, marker='o', label="Confidence")
    plt.xlabel("Confidence")
    plt.ylabel("Accuracy")
    plt.title("Reliability Diagram")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_reliability(y_true, y_prob)
